# End to End Benchmark: Producer (Dask) - Kafka Cluster - Consumer (Spark)

In [1]:
%%capture
# System Libraries
import sys, os
sys.path.insert(0, "..")
import pandas as pd

## logging
import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger().setLevel(logging.ERROR)
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("radical.utils").setLevel(logging.ERROR)
 
# Pilot-Streaming
import pilot.streaming
import masa.spark
import mass.kafka

/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:72: UserWarning: logging handlers already registered.
  warnings.warn('logging handlers already registered.')


In [2]:
num_broker_nodes=1
num_producer_nodes=1
number_spark_nodes=1
run_id = "1"
topic_name = "test" + run_id
number_parallel_tasks=1

kafka_pilot_description1 = {
                    "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
                    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
                    "number_cores": 48*num_broker_nodes,
                    "project": "TG-MCB090174",
                    "queue": "normal",
                    "walltime": 159,
                    "type":"kafka"
                }
kafka_pilot = pilot.streaming.PilotComputeService.create_pilot(kafka_pilot_description1)
kafka_pilot.wait()

dask_pilot_description = {
    "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48*num_producer_nodes,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 159,
    "type":"dask"
}
dask_pilot = pilot.streaming.PilotComputeService.create_pilot(dask_pilot_description)
dask_pilot.wait()

spark_pilot_description = {
    "resource":"slurm+ssh://login1.wrangler.tacc.utexas.edu",
    "working_directory": os.path.join('/work/01131/tg804093/wrangler/', "work"),
    "number_cores": 48*number_spark_nodes,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 159,
    "type":"spark"
}
spark_pilot = pilot.streaming.PilotComputeService.create_pilot(spark_pilot_description)
spark_pilot.wait()

**** Job: 60324 State : Pending
look for configs in: /work/01131/tg804093/wrangler/work/kafka-a839af1c-fa69-11e7-b88a-549f3509766c/config
['broker-0']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a839af1c-fa69-11e7-b88a-549f3509766c/config (Mon Jan 15 21:02:15 2018)
{'zookeeper.connection.timeout.ms': '6000', 'broker.id': '0', 'listeners': 'PLAINTEXT://c251-133:9092', 'zookeeper.connect': 'c251-133:2181'}
**** Job: 60325 State : Pending
init distributed client
init distributed client
**** Job: 60326 State : Pending
Create Spark Context for URL: spark://129.114.58.136:7077
Create Spark Context for URL: spark://129.114.58.136:7077


In [4]:
prod=mass.kafka.MiniApp(
                           dask_scheduler=dask_pilot.get_details()['master_url'],
                           kafka_zk_hosts=kafka_pilot.get_details()["master_url"],
                           number_parallel_tasks=number_parallel_tasks,
                           number_clusters=192, # kmeans
                           number_points_per_cluster=520834, #52084, # kmeans
                           number_points_per_message=10000, # kmeans
                           number_dim=3, # kmeans
                           number_messages=6400, # light
                           number_produces=1,
                           number_partitions=num_broker_nodes*12,
                           topic_name=topic_name,
                           application_type = "kmeans"
                        )
prod.run_in_background()              

look for configs in: /work/01131/tg804093/wrangler/work/kafka-a839af1c-fa69-11e7-b88a-549f3509766c/config
['broker-0']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a839af1c-fa69-11e7-b88a-549f3509766c/config (Mon Jan 15 21:02:15 2018)
{'zookeeper.connection.timeout.ms': '6000', 'broker.id': '0', 'listeners': 'PLAINTEXT://c251-133:9092', 'zookeeper.connect': 'c251-133:2181'}
Kafka: c251-133:2181, Dask: tcp://129.114.58.135:8786, Number Dask Nodes: 1,  Number Parallel Producers: 1
/home/01131/tg804093/work/kafka_2.11-1.0.0/bin/kafka-topics.sh --delete --zookeeper c251-133:2181 --topic test1
/home/01131/tg804093/work/kafka_2.11-1.0.0/bin/kafka-topics.sh --create --zookeeper c251-133:2181 --replication-factor 1 --partitions 12 --topic test1
/home/01131/tg804093/work/kafka_2.11-1.0.0/bin/kafka-topics.sh --describe --zookeeper c251-133:2181 --topic test1
Generate Block ID: 0
Waiting for Dask Tasks to complete


In [5]:
consumer = masa.spark.MiniApp(
                          spark_master=spark_pilot.get_details()["master_url"],
                          kafka_zk_hosts=kafka_pilot.get_details()["master_url"],
                          topic_name = topic_name,
                          scenario="1-1-1"
                         )
consumer.run_in_background()

Create Spark Context for URL: spark://129.114.58.136:7077
look for configs in: /work/01131/tg804093/wrangler/work/kafka-a839af1c-fa69-11e7-b88a-549f3509766c/config
['broker-0']
Kafka Config: /work/01131/tg804093/wrangler/work/kafka-a839af1c-fa69-11e7-b88a-549f3509766c/config (Mon Jan 15 21:02:15 2018)
{'zookeeper.connection.timeout.ms': '6000', 'broker.id': '0', 'listeners': 'PLAINTEXT://c251-133:9092', 'zookeeper.connect': 'c251-133:2181'}
/home/01131/tg804093/work/kafka_2.11-1.0.0/bin/kafka-topics.sh --describe --topic test1 --zookeeper c251-133:2181
Streaming Window: 60 sec
spark-submit --master spark://129.114.58.136:7077 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.1 --conf 'spark.executor.memory=110g' /home/01131/tg804093/anaconda2/lib/python2.7/site-packages/masa/spark.py spark://129.114.58.136:7077 c251-133:2181 test1 1-1-1 60


In [ ]:
mass.wait()
consumer.cancel()

In [ ]:
%%capture
kafka_pilot.cancel()
dask_pilot.cancel()
spark_pilot.cancel()

tornado.application - ERROR - Exception in callback <functools.partial object at 0x2aac573edba8>
Traceback (most recent call last):
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 626, in _discard_future_result
    future.result()
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/tornado/gen.py", line 1063, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/01131/tg804093/anaconda2/lib/python2.7/site-packages/distributed/client.py", line 804, in _update_scheduler_info
    self._scheduler_iden